In [1]:
import csv
import logging
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException, ElementClickInterceptedException, NoSuchFrameException
from urllib.parse import urlparse
import concurrent.futures

logging.basicConfig(level=logging.INFO)

def validity_check(url):
    try:
        output = urlparse(url)
        return all([output.scheme, output.netloc])
    except ValueError:
        return False

def configure_browser_options(headless=True):
    options = Options()
    if headless:
        options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--disable-extensions")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-images")
    options.page_load_strategy = 'eager'
    return options

def handle_cookies(driver):
    cookie_buttons = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT","AGREE","Accept all cookies","Accept All Cookies","Allow","ALLOW","OK","Ok","I accept","I Accept","I ACCEPT",
        "Accept Cookies","ACCEPT COOKIES","Allow all cookies"
    ]
    for button_text in cookie_buttons:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[contains(translate(text(),'ACCEPT','accept'),'{button_text.lower()}')] | //a[contains(translate(text(),'ACCEPT','accept'),'{button_text.lower()}')]"))
            )
            accept_button.click()
            break
        except (NoSuchElementException, TimeoutException, ElementClickInterceptedException):
            pass

def find_login_button(driver):
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN"]
    for login_pat in login_section:
        try:
            login_button = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
            )
            actions = ActionChains(driver)
            actions.move_to_element(login_button).perform()
            try:
                login_button.click()
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
            return True
        except TimeoutException:
            continue
    return False

def switch_to_iframe(driver):
    try:
        WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
    except TimeoutException:
        pass

def check_google_oidc(driver, website_url):
    google_oidc_patterns = [
        "Sign in with Google", "Sign In with Google", "Continue with Google", "Login with Google",
        "Sign in using Google", "Log In with Google", "Login using Google",
        "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE"
    ]

    oidc_method = "N/A"
    redirect_or_element_found = False
    text_pattern_found = False

    try:
        WebDriverWait(driver, 10).until(
            lambda x: driver.current_url != website_url or driver.find_element(By.XPATH, "//a[contains(@href,'google')] | //button[contains(@href,'google')] | //div[contains(@href,'google')]")
        )
        redirect_or_element_found = True
    except TimeoutException:
        pass

    if any(pattern in driver.page_source for pattern in google_oidc_patterns):
        text_pattern_found = True

    if redirect_or_element_found:
        if len(driver.window_handles) > 1:
            oidc_method = "Pop-up"
        elif driver.current_url != website_url:
            oidc_method = "Redirect URL"
        else:
            oidc_method = "Direct Link/Button"
    elif text_pattern_found:
        oidc_method = "Text Pattern"

    has_google_oidc = oidc_method != "N/A"
    return has_google_oidc, oidc_method

def check_oidc(website_url, headless=True):
    try:
        driver = webdriver.Chrome(options=configure_browser_options(headless))

        if not validity_check(website_url):
            logging.error(f"Invalid URL: {website_url}")
            return False, "N/A"

        driver.get(website_url)
        handle_cookies(driver)
        if not find_login_button(driver):
            logging.warning("Login button not found")
        switch_to_iframe(driver)

        has_google_oidc, method = check_google_oidc(driver, website_url)

        driver.quit()
        return has_google_oidc, method

    except Exception as e:
        logging.error(f"Error analyzing {website_url}: {e}")
        return False, "Error"

def check_oidc_concurrent(website_url):
    return (website_url, *check_oidc(website_url))

def main(websites):
    num_threads = 10  # Adjust based on your requirements and capabilities

    with concurrent.futures.ThreadPoolExecutor(max_workers=num_threads) as executor:
        future_to_url = {executor.submit(check_oidc_concurrent, url): url for url in websites}
        for future in concurrent.futures.as_completed(future_to_url):
            url = future_to_url[future]
            try:
                website, uses_google_oidc, method = future.result()
                print(f"{website} uses Google OIDC: {uses_google_oidc}, Method: {method}")
            except Exception as exc:
                print(f"{url} generated an exception: {exc}")

if __name__ == "__main__":
    websites = [
'https://indianexpress.com/','https://www.fandom.com/','https://www.aliexpress.com/','https://www.imdb.com/','https://www.doordash.com/',
    'https://www.studocu.com/en-gb','https://shopee.ph/','https://www.veed.io/','https://www.coursehero.com/',
    'https://www.marketwatch.com/','https://cloudinary.com/','https://www.crazygames.com/','https://groww.in/',
    'https://www.mobile.de/','https://www.vitalsource.com/','https://www.gitbook.com/','https://disqus.com/',
    'https://www.mongodb.com/','https://apkpure.com/','https://brainly.com/','https://www.duplichecker.com/',
    'https://www.bhphotovideo.com/','https://www.blizzard.com/en-gb/','https://www.fotor.com/','https://www.abc.net.au/',
    'https://www.repubblica.it/','https://www.pdf2go.com/','https://www.drive2.ru/','https://www.cbssports.com/',
    'https://www.turnitin.com/','https://www.mirror.co.uk/','https://tvtropes.org/','https://www.skyscanner.net/',
    'https://www.hindustantimes.com/','https://www.imdb.com/'
    ]
    main(websites)


https://www.w3schools.com/ uses Google OIDC: True, Method: Redirect URL


https://vimeo.com/ uses Google OIDC: True, Method: Redirect URL
https://medium.com/ uses Google OIDC: True, Method: Direct Link/Button
https://www.shopify.com/ uses Google OIDC: False, Method: N/A
https://www.researchgate.net/ uses Google OIDC: True, Method: Redirect URL
https://www.chess.com/ uses Google OIDC: True, Method: Redirect URL


https://pixabay.com/re uses Google OIDC: False, Method: N/A


https://indianexpress.com/ uses Google OIDC: False, Method: N/A
https://www.fandom.com/ uses Google OIDC: False, Method: N/A
https://www.aliexpress.com/ uses Google OIDC: True, Method: Redirect URL


https://www.imdb.com/ uses Google OIDC: False, Method: N/A


...

https://www.mirror.co.uk/ uses Google OIDC: False, Method: N/A


ERROR:root:Error analyzing https://www.skyscanner.net/: Message: invalid argument: missing 'ELEMENT'
  (Session info: headless chrome=119.0.6045.124)
Stacktrace:
	GetHandleVerifier [0x00007FF65ED882B2+55298]
	(No symbol) [0x00007FF65ECF5E02]
	(No symbol) [0x00007FF65EBB05AB]
	(No symbol) [0x00007FF65EC2C079]
	(No symbol) [0x00007FF65EC120AA]
	(No symbol) [0x00007FF65EC2AAA4]
	(No symbol) [0x00007FF65EC11E83]
	(No symbol) [0x00007FF65EBE670A]
	(No symbol) [0x00007FF65EBE7964]
	GetHandleVerifier [0x00007FF65F100AAB+3694587]
	GetHandleVerifier [0x00007FF65F15728E+4048862]
	GetHandleVerifier [0x00007FF65F14F173+4015811]
	GetHandleVerifier [0x00007FF65EE247D6+695590]
	(No symbol) [0x00007FF65ED00CE8]
	(No symbol) [0x00007FF65ECFCF34]
	(No symbol) [0x00007FF65ECFD062]
	(No symbol) [0x00007FF65ECED3A3]
	BaseThreadInitThunk [0x00007FFCF87D257D+29]
	RtlUserThreadStart [0x00007FFCF916AA58+40]



https://www.skyscanner.net/ uses Google OIDC: False, Method: Error


https://www.hindustantimes.com/ uses Google OIDC: False, Method: N/A


https://www.imdb.com/ uses Google OIDC: False, Method: N/A


In [2]:
import concurrent.futures
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException

def configure_browser_options():
    options = Options()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument('--disable-gpu')
    options.add_argument("--window-size=1920,1080")
    options.add_argument('--disable-extensions')
    options.add_argument('--disable-infobars')
    options.add_argument('--disable-browser-side-navigation')
    options.add_argument('--disable-setuid-sandbox')
    options.add_argument('--remote-debugging-port=9222')
    return options

def accept_cookies(driver):
    cookie_buttons_texts = [
        "Accept all & visit the site", "Accept", "Agree", "Close", "Ignore", 
        "Agree & Continue", "Agree & Close", "AGREE & CLOSE", "CONSENT", 
        "Consent", "ACCEPT", "AGREE", "Accept all cookies", "Accept All Cookies",
        "Allow", "ALLOW", "OK", "Ok", "I accept", "I Accept", "I ACCEPT",
        "Accept Cookies", "ACCEPT COOKIES", "Allow all cookies"
    ]
    for button_text in cookie_buttons_texts:
        try:
            accept_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, f"//button[contains(., '{button_text}')] | //a[contains(., '{button_text}')] | //div[contains(., '{button_text}')]"))
            )
            accept_button.click()
            print(f"Clicked cookie button: {button_text}")
            break
        except TimeoutException:
            continue

def handle_popups_and_iframes(driver):
    try:
        WebDriverWait(driver, 5).until(EC.number_of_windows_to_be(2))
        driver.switch_to.window(driver.window_handles[1])
        print("Switched to pop-up window")
    except TimeoutException:
        print("No pop-up window detected")
    
    try:
        WebDriverWait(driver, 5).until(EC.frame_to_be_available_and_switch_to_it((By.TAG_NAME, "iframe")))
        print("Switched to iframe")
    except TimeoutException:
        print("No iframe detected")

def find_and_click_login_button(driver):
    login_section = ["Login", "Log in", "Sign in", "Sign In", "Log In", "LOGON", "SIGN IN", "LOGIN", "LOG IN"]
    for login_pat in login_section:
        try:
            login_button = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.PARTIAL_LINK_TEXT, login_pat))
            )
            actions = ActionChains(driver)
            actions.move_to_element(login_button).perform()
            try:
                login_button.click()
                return True
            except ElementClickInterceptedException:
                driver.execute_script("arguments[0].click();", login_button)
                return True
        except TimeoutException:
            continue

    for text in login_section:
        try:
            login_button = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, f"//*[contains(text(), '{text}')]"))
            )
            login_button.click()
            return True
        except TimeoutException:
            continue

    return False

def check_website(url):
    try:
        chrome_options = configure_browser_options()
        driver = webdriver.Chrome(options=chrome_options)
        driver.get(url)
        # Accept cookies
        accept_cookies(driver)

        # Find and click login/sign-in button
        login_found = find_and_click_login_button(driver)

        # Handle pop-ups and iframes
        handle_popups_and_iframes(driver)

        # Check for Google login option
        google_login_present = False
        if login_found:
            for text in ["Sign in with Google", "Sign In with Google", "Continue with Google", "Login with Google", "Sign in using Google", "Log In with Google",
                         "Login using Google", "LOG IN using Google", "LOG IN with Google", "LOG IN WITH GOOGLE"]:
                if text in driver.page_source:
                    google_login_present = True
                    break

        driver.quit()
        return url, google_login_present

    except Exception as e:
        print(f"Error processing {url}: {e}")
        return url, False
        
# Function for concurrent checking of websites
def check_oidc_concurrent(website_url):
    return (website_url, *check_oidc(website_url))

# Main function to execute the script
def main(websites):
    num_threads = 8

def main(websites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(check_website, url): url for url in websites}
        for future in concurrent.futures.as_completed(future_to_url):
            url, result = future.result()
            print(f"Website: {url}, Google Login Present: {result}")

if __name__ == "__main__":
    websites = ['https://www.shopify.com/','https://www.w3schools.com/','https://medium.com/',
                'https://www.researchgate.net/','https://vimeo.com/','https://pixabay.com/re',
                'https://www.chess.com/','https://indianexpress.com/','https://www.fandom.com/',
                'https://www.aliexpress.com/','https://www.imdb.com/','https://www.doordash.com/',
                'https://www.studocu.com/en-gb','https://shopee.ph/','https://www.veed.io/',
                'https://www.coursehero.com/','https://www.marketwatch.com/','https://cloudinary.com/',
                'https://www.crazygames.com/','https://groww.in/','https://www.mobile.de/',
                'https://www.vitalsource.com/','https://www.gitbook.com/','https://disqus.com/',
                'https://www.mongodb.com/','https://apkpure.com/','https://brainly.com/',
                'https://www.duplichecker.com/','https://www.bhphotovideo.com/',
                'https://www.blizzard.com/en-gb/','https://www.fotor.com/','https://www.abc.net.au/',
                'https://www.repubblica.it/', 'https://www.pdf2go.com/','https://www.drive2.ru/', 'https://www.cbssports.com/',
                'https://www.turnitin.com/','https://www.mirror.co.uk/','https://tvtropes.org/','https://www.skyscanner.net/',
                'https://www.hindustantimes.com/','https://www.imdb.com/']
    main(websites)


Error processing https://www.shopify.com/: Message: stale element reference: stale element not found
  (Session info: headless chrome=119.0.6045.124); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF65ED882B2+55298]
	(No symbol) [0x00007FF65ECF5E02]
	(No symbol) [0x00007FF65EBB05AB]
	(No symbol) [0x00007FF65EBBE59A]
	(No symbol) [0x00007FF65EBB59E9]
	(No symbol) [0x00007FF65EBB4298]
	(No symbol) [0x00007FF65EBB6FDF]
	(No symbol) [0x00007FF65EBB70D0]
	(No symbol) [0x00007FF65EBF3265]
	(No symbol) [0x00007FF65EBE8586]
	(No symbol) [0x00007FF65EC120AA]
	(No symbol) [0x00007FF65EBE80CF]
	(No symbol) [0x00007FF65EC122C0]
	(No symbol) [0x00007FF65EC2AAA4]
	(No symbol) [0x00007FF65EC11E83]
	(No symbol) [0x00007FF65EBE670A]
	(No symbol) [0x00007FF65EBE7964]
	GetHandleVerifier [0x00007FF65F100AAB+3694587]
	GetHandleVerifier [0x00007FF65F15728E+4048862

Error processing https://pixabay.com/re: Message: disconnected: unable to connect to renderer
Stacktrace:
	GetHandleVerifier [0x00007FF65ED882B2+55298]
	(No symbol) [0x00007FF65ECF5E02]
	(No symbol) [0x00007FF65EBB05AB]
	(No symbol) [0x00007FF65EBE3A04]
	(No symbol) [0x00007FF65EBDF13C]
	(No symbol) [0x00007FF65EBDB30E]
	(No symbol) [0x00007FF65EC1A198]
	(No symbol) [0x00007FF65EC19CB0]
	(No symbol) [0x00007FF65EC11E83]
	(No symbol) [0x00007FF65EBE670A]
	(No symbol) [0x00007FF65EBE7964]
	GetHandleVerifier [0x00007FF65F100AAB+3694587]
	GetHandleVerifier [0x00007FF65F15728E+4048862]
	GetHandleVerifier [0x00007FF65F14F173+4015811]
	GetHandleVerifier [0x00007FF65EE247D6+695590]
	(No symbol) [0x00007FF65ED00CE8]
	(No symbol) [0x00007FF65ECFCF34]
	(No symbol) [0x00007FF65ECFD062]
	(No symbol) [0x00007FF65ECED3A3]
	BaseThreadInitThunk [0x00007FFCF87D257D+29]
	RtlUserThreadStart [0x00007FFCF916AA58+40]

Clicked cookie button: Accept
Website: https://www.aliexpress.com/, Google Login Present: F

Website: https://www.marketwatch.com/, Google Login Present: False
Switched to iframe
Error processing https://cloudinary.com/: Message: disconnected: unable to connect to renderer
Stacktrace:
	GetHandleVerifier [0x00007FF65ED882B2+55298]
	(No symbol) [0x00007FF65ECF5E02]
	(No symbol) [0x00007FF65EBB05AB]
	(No symbol) [0x00007FF65EBE3A04]
	(No symbol) [0x00007FF65EBDF13C]
	(No symbol) [0x00007FF65EBDB30E]
	(No symbol) [0x00007FF65EC1A198]
	(No symbol) [0x00007FF65EC19CB0]
	(No symbol) [0x00007FF65EC11E83]
	(No symbol) [0x00007FF65EBE670A]
	(No symbol) [0x00007FF65EBE7964]
	GetHandleVerifier [0x00007FF65F100AAB+3694587]
	GetHandleVerifier [0x00007FF65F15728E+4048862]
	GetHandleVerifier [0x00007FF65F14F173+4015811]
	GetHandleVerifier [0x00007FF65EE247D6+695590]
	(No symbol) [0x00007FF65ED00CE8]
	(No symbol) [0x00007FF65ECFCF34]
	(No symbol) [0x00007FF65ECFD062]
	(No symbol) [0x00007FF65ECED3A3]
	BaseThreadInitThunk [0x00007FFCF87D257D+29]
	RtlUserThreadStart [0x00007FFCF916AA58+40]

Webs

Website: https://www.bhphotovideo.com/, Google Login Present: False
Error processing https://brainly.com/: Message: disconnected: unable to connect to renderer
Stacktrace:
	GetHandleVerifier [0x00007FF65ED882B2+55298]
	(No symbol) [0x00007FF65ECF5E02]
	(No symbol) [0x00007FF65EBB05AB]
	(No symbol) [0x00007FF65EBE3A04]
	(No symbol) [0x00007FF65EBDF13C]
	(No symbol) [0x00007FF65EBDB30E]
	(No symbol) [0x00007FF65EC1A198]
	(No symbol) [0x00007FF65EC19CB0]
	(No symbol) [0x00007FF65EC11E83]
	(No symbol) [0x00007FF65EBE670A]
	(No symbol) [0x00007FF65EBE7964]
	GetHandleVerifier [0x00007FF65F100AAB+3694587]
	GetHandleVerifier [0x00007FF65F15728E+4048862]
	GetHandleVerifier [0x00007FF65F14F173+4015811]
	GetHandleVerifier [0x00007FF65EE247D6+695590]
	(No symbol) [0x00007FF65ED00CE8]
	(No symbol) [0x00007FF65ECFCF34]
	(No symbol) [0x00007FF65ECFD062]
	(No symbol) [0x00007FF65ECED3A3]
	BaseThreadInitThunk [0x00007FFCF87D257D+29]
	RtlUserThreadStart [0x00007FFCF916AA58+40]

Website: https://brainly.

Website: https://www.turnitin.com/, Google Login Present: False
Error processing https://tvtropes.org/: Message: disconnected: unable to connect to renderer
Stacktrace:
	GetHandleVerifier [0x00007FF65ED882B2+55298]
	(No symbol) [0x00007FF65ECF5E02]
	(No symbol) [0x00007FF65EBB05AB]
	(No symbol) [0x00007FF65EBE3A04]
	(No symbol) [0x00007FF65EBDF13C]
	(No symbol) [0x00007FF65EBDB30E]
	(No symbol) [0x00007FF65EC1A198]
	(No symbol) [0x00007FF65EC19CB0]
	(No symbol) [0x00007FF65EC11E83]
	(No symbol) [0x00007FF65EBE670A]
	(No symbol) [0x00007FF65EBE7964]
	GetHandleVerifier [0x00007FF65F100AAB+3694587]
	GetHandleVerifier [0x00007FF65F15728E+4048862]
	GetHandleVerifier [0x00007FF65F14F173+4015811]
	GetHandleVerifier [0x00007FF65EE247D6+695590]
	(No symbol) [0x00007FF65ED00CE8]
	(No symbol) [0x00007FF65ECFCF34]
	(No symbol) [0x00007FF65ECFD062]
	(No symbol) [0x00007FF65ECED3A3]
	BaseThreadInitThunk [0x00007FFCF87D257D+29]
	RtlUserThreadStart [0x00007FFCF916AA58+40]

Error processing https://www